In [1]:
%load_ext autoreload
%autoreload 2

from code_py.DIAMOnD import *
from code_py.backbone import Human_Genes_Graph_Analysis
import markov_clustering as mc
from joblib import Parallel, delayed
import statistics 
from tqdm import tqdm

sys_path = '/Users/tanselsimsek/Desktop/DS/2.1/BI/Final_project-1/'


#  Part 1: Preprocessing

In [2]:
disease_code = "C0020796"
hga = Human_Genes_Graph_Analysis(sys_path,disease_ID = disease_code)

#### 1.1 -- Filtering and revoving self-loops

In [3]:
# Preprocessing the Biogrid dataset by filtering Homo Sapiens, removing duplicated and self loops
hs_putative_genes = hga.preprocessing_dataset(homo_sap=True,drop_duplicates=True,remove_self_loops=True)

Number of putative genes: 889884


#### 1.2 -- Filtering Disease Genes

In [4]:
hs_disease_genes_df,hs_disease_genes = hga.query_disease_genes()

Found 139 disease genes in Profound Mental Retardation


#### 1.3 -- Creating LCC sub-graph,adjacency matrix,

In [5]:
# We are creating the graph with nx from the from the filtered PPI dataset and returning the sub graph, adj matrix, nodes and edges of LCC 
pgenes_sub_graph,pgenes_adj,pnodes,pedges = hga.LCC_to_adj(hs_putative_genes)

# of connected components: 1
19618
Name: 
Type: Graph
Number of nodes: 19618
Number of edges: 665061
Average degree:  67.8011


#### 1.3 -- Cross Validation

In [6]:
ds_genes_train,ds_genes_test = hga.KFold_CV(hs_disease_genes,n_folds=5,shuffle_flag=True)

#### 1.4 -- Saving LCC and Seed Genes to files for Cytoscape

In [14]:
np.savetxt("LCC .csv", 
            np.array(pgenes_sub_graph.edges),
            delimiter =", ", 
            fmt ='% s')

In [9]:
for i in range(len(ds_genes_test)):
    np.savetxt("test"+str(i)+".csv", 
            ds_genes_test[i],
            delimiter =", ", 
            fmt ='% s')
    np.savetxt("train"+str(i)+".csv", 
            ds_genes_train[i],
            delimiter =", ", 
            fmt ='% s')

# Part 2: Algorithms  

### 2.1 -- MCL Algorithm 

In [ ]:
# Applying MLC Algoritm by given inflation range (1.5, 2.7, step = 0.1)
results = Parallel(n_jobs=3)(delayed(hga.MCL)(pgenes_adj,i) for i in tqdm(np.arange(1.5,2.7,0.1)))
hga.list_to_pikle(results,'MLC_modularity')

In [8]:
results_list_from_pkl = hga.read_pickle_list('MLC_modularity')
for i in enumerate(np.arange(1.5,2.7,0.1)):
    print("inflation:", round(i[1],2), "modularity:", results_list_from_pkl[i[0]])
    

inflation: 1.8 modularity: 1
inflation: 1.9 modularity: 2
inflation: 2.0 modularity: 3
inflation: 2.1 modularity: 4
inflation: 2.2 modularity: 5
inflation: 2.3 modularity: 6
inflation: 2.4 modularity: 7
inflation: 2.5 modularity: 8
inflation: 2.6 modularity: 9


### 2.1.1 -- Creating Clusters

In [7]:
best_inflation = 1.8
result = mc.run_mcl(pgenes_adj, inflation=best_inflation)
clusters = mc.get_clusters(result)
print(str(len(clusters))+" of clusters obtained with inflation of "+str(best_inflation))

2106 of clusters obtained with inflation of 1.8


In [8]:
_, enriched_genes,enriched_cluster_ID = hga.MLC_eval(pgenes_sub_graph,ds_genes_train,clusters)

Fold number:  0
6 disease genes in cluster 0 --> 0.165607
35 disease genes in cluster 2 --> 0.013666
3 disease genes in cluster 6 --> 0.217906
6 disease genes in cluster 11 --> 0.165457
3 disease genes in cluster 27 --> 0.034026
4 disease genes in cluster 46 --> 0.154399
7 disease genes in cluster 53 --> 0.115389
3 disease genes in cluster 68 --> 0.127471
Fold number:  1
5 disease genes in cluster 0 --> 0.164454
40 disease genes in cluster 2 --> 0.001084
3 disease genes in cluster 6 --> 0.217906
6 disease genes in cluster 11 --> 0.165457
8 disease genes in cluster 53 --> 0.073821
3 disease genes in cluster 67 --> 0.22377
3 disease genes in cluster 68 --> 0.127471
Fold number:  2
3 disease genes in cluster 0 --> 0.087338
34 disease genes in cluster 2 --> 0.020074
4 disease genes in cluster 6 --> 0.136394
7 disease genes in cluster 11 --> 0.139195
10 disease genes in cluster 53 --> 0.020798
3 disease genes in cluster 67 --> 0.22377
3 disease genes in cluster 68 --> 0.127471
Fold number: 

In [10]:
hga.MCL_evaluation_metrics(pgenes_sub_graph,ds_genes_test,clusters,enriched_cluster_ID)

TP: 51 --- FP: 23844 --- FN: 14839
Precision: 0.002134 --- Recall: 0.003425 --- F1 Score: 0.00263


### 2.2 -- DIAMOnD Algorithm

In [15]:
precision = []
recall = []
f1_score = []
for i in range(0,5):
    added_nodes, predicted_nodes = DIAMOnD(G_original=pgenes_sub_graph,
                            seed_genes=ds_genes_train[i],
                            max_number_of_added_nodes=len(ds_genes_train[i]),alpha=1)
    #TP numero di geni che effettivamente sono disease genes
    TP = len(set(predicted_nodes).intersection(set(ds_genes_test[i])))
    #numero di geni riportati come veri ma che non sono disease genes 
    FP = len(ds_genes_train[i]) - TP
    FN = len(ds_genes_test[i]) - TP
    precision.append(TP/(TP+FP))
    recall.append((TP)/(TP+FN))
    try:
        f1_score.append((2*precision[i]*recall[i])/(precision[i]+recall[i]))
    except:
        print("zero division")


print("Precision: " + str(round(statistics.mean(precision),6)) + " ± " +str(round(statistics.stdev(precision),6)))
print("Recall: " + str(round(statistics.mean(recall),6)) + " ± " +str(round(statistics.stdev(recall),6)))
print("F1 Score: " + str(round(statistics.mean(f1_score),6)) + " ± " +str(round(statistics.stdev(f1_score),6)))


DIAMOnD(): ignoring 68 of 323 seed genes that are not in the network
DIAMOnD(): ignoring 72 of 323 seed genes that are not in the network
DIAMOnD(): ignoring 69 of 323 seed genes that are not in the network
DIAMOnD(): ignoring 68 of 323 seed genes that are not in the network
DIAMOnD(): ignoring 63 of 324 seed genes that are not in the network
Precision: 0.012997 ± 0.003399
Recall: 0.051944 ± 0.013398
F1 Score: 0.020792 ± 0.005423


### 2.3 -- DiaBLE Algorithm

In [ ]:
precision_diable = []
recall_diable = []
f1_score_diable = []
for i in range(0,5):
    added_nodes, predicted_nodes = DIAMOnD(G_original=pgenes_sub_graph,
                                           seed_genes=ds_genes_train[i],
                                           max_number_of_added_nodes=len(ds_genes_train[i]),
                                           alpha=1,DiaBLE=True)
                                           
    #TP numero di geni che effettivamente sono disease genes
    TP = len(set(predicted_nodes).intersection(set(ds_genes_test[i])))
    #numero di geni riportati come veri ma che non sono disease genes 
    FP = len(ds_genes_train[i]) - TP
    #numero di geni che 
    FN = len(ds_genes_train[i]) - TP
    precision_diable.append(TP/(TP+FP))
    recall_diable.append((TP)/(TP+FN))
    try:
        f1_score_diable.append((2*precision[i]*recall[i])/(precision[i]+recall[i]))
    except:
        print("zero division")


print("Precision: " + str(round(statistics.mean(precision_diable),6)) + " ± " +str(round(statistics.stdev(precision_diable),6)))
print("Recall: " + str(round(statistics.mean(recall_diable),6)) + " ± " +str(round(statistics.stdev(recall_diable),6)))
print("F1 Score: " + str(round(statistics.mean(f1_score_diable),6)) + " ± " +str(round(statistics.stdev(f1_score_diable),6)))


### 2.6 -- Random Walk with Restart 

In [ ]:
for fold in tqdm(range(0,5)):
    rwr_enriched_genes = hga.RWR(pgenes_sub_graph,ds_genes_train[fold],max_print_items=0)
    break

# Part 3: Extented Validation  

In [27]:
all_gene_disease = hga.query_disease_genes_extendend()